In [ ]:
filepath = %pwd
filepath

In [ ]:
from os import path
fname = path.expanduser(str(filepath)+'/track.csv')

In [ ]:
path.getsize(fname)

In [ ]:
path.getsize(fname) / (1<<10)

In [ ]:
!head "$fname"

In [ ]:
with open(fname) as fp:
    for lnum, line in enumerate(fp):
        if lnum > 10:
            break
        print(line[:-1])

In [ ]:
!wc -l "$fname"

In [ ]:
with open(fname) as fp:
    print(sum(1 for line in fp))

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(fname)

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df = pd.read_csv(fname, parse_dates=['time'])

In [ ]:
df.dtypes

In [ ]:
df['lat']

In [ ]:
df.lat

In [ ]:
df[['lat', 'lng']]

In [ ]:
df['lat'][0]

In [ ]:
df.loc[0]

In [ ]:
df.loc[2:7]

In [ ]:
df[['lat', 'lng']][2:7]

In [ ]:
df.index

In [ ]:
import numpy as np
df1 = pd.DataFrame(np.arange(10).reshape((5,2)), columns=['x', 'y'], index=['a', 'b', 'c', 'd', 'e'])
df1

In [ ]:
df1.loc['a']

In [ ]:
df1.loc['b': 'd']

In [ ]:
df.index

In [ ]:
df.index = df['time']
df.index

In [ ]:
df.loc['2015-08-20 04:18:54']

In [ ]:
df.loc['2015-08-20 03:48']

In [ ]:
import pytz

In [ ]:
ts = df.index[0]

In [ ]:
ts.tz_localize(pytz.UTC)

In [ ]:
ts.tz_localize(pytz.UTC).tz_convert(pytz.timezone('Asia/Jerusalem'))

In [ ]:
df.index = df.index.tz_localize(pytz.UTC).tz_convert(pytz.timezone('Asia/Jerusalem'))
df.index[:10]

In [ ]:
%pwd

In [ ]:
import geo

In [ ]:
import sys
sys.path

In [ ]:
??geo

In [ ]:
from geo import circle_dist

In [ ]:
lat1, lng1 = df.iloc[0].lat, df.iloc[0].lng
lat2, lng2 = df.iloc[1].lat, df.iloc[1].lng

In [ ]:
circle_dist(lat1, lng1, lat2, lng2)

In [ ]:
s = pd.Series(np.arange(5))

In [ ]:
s

In [ ]:
s.shift()

In [ ]:
s.shift(-1)

In [ ]:
dist = circle_dist(df['lat'], df['lng'], df['lat'].shift(), df['lng'].shift())

In [ ]:
dist[:10]

In [ ]:
dist.sum()

In [ ]:
dt = df['time'] - df['time'].shift()

In [ ]:
dt[:10]

In [ ]:
dt.sum()

In [ ]:
dt[1].total_seconds()

In [ ]:
dt[1] / np.timedelta64(1, 'h')

In [ ]:
dt[1].total_seconds()/3600

In [ ]:
speed = dist / (dt / np.timedelta64(1, 'h'))

In [ ]:
speed[:10]

In [ ]:
df['dist'] = dist
df['dt'] = dt

In [ ]:
df1m = df.resample('1min').sum()

In [ ]:
df1m.index

In [ ]:
df1m.columns

In [ ]:
df['dt'] = dt / np.timedelta64(1, 'h')
df1m = df.resample('1min').sum()
speed1m = df1m['dist'] / df1m['dt']

In [ ]:
speed1m[:10]

In [ ]:
speed1m.describe()

In [ ]:
%matplotlib inline

In [ ]:
speed1m.plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
plt.style.use('seaborn-whitegrid')

In [ ]:
speed1m.plot()

In [ ]:
speed1m.plot.box()

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[df['lng'].mean(), df['lat'].mean()], zoom_start=15)
mdf = df.resample('T').mean()

def add_marker(row):
    marker = folium.CircleMarker([row['lng'], row['lat']], radius=5, color='red', popup=row.name.strftime('%H:%M'))
    marker.add_to(m)

mdf.apply(add_marker, axis=1)
m

In [ ]:
mdf.columns

In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
pt = Point(1, 2)

In [ ]:
pt

In [ ]:
pt.x

In [ ]:
poly = Polygon([[0, 0], [0, 10], [10, 10], [10, 0]])

In [ ]:
poly.area

In [ ]:
poly.centroid.xy

In [ ]:
poly.intersects(pt)

In [ ]:
poly.intersects(Point(10, 20))

In [ ]:
mdf['pt'] = mdf[['lng', 'lat']].apply(Point, axis=1)
mdf.head()

In [ ]:
mean_lng, max_lng = mdf['lng'].mean(), mdf['lng'].max()
mean_lat, max_lat = mdf['lat'].mean(), mdf['lat'].max()

In [ ]:
poly = Polygon([
    [mean_lng, mean_lat],
    [mean_lng, max_lat],
    [max_lng, max_lat],
    [max_lng, mean_lat]
])

In [ ]:
poly.exterior.xy

In [ ]:
import numpy as np

In [ ]:
np.stack(poly.exterior.xy).T

In [ ]:
m = folium.Map(location=[df['lng'].mean(), df['lat'].mean()], zoom_start=15)
mdf = df.resample('T').mean()
mdf['pt'] = mdf[['lng', 'lat']].apply(Point, axis=1)

def add_marker(row):
    color = 'yellow' if poly.intersects(row['pt']) else 'green'
    marker = folium.CircleMarker([row['lng'], row['lat']], radius=5, color='red', popup=row.name.strftime('%H:%M'),
                                fill_color=color)
    marker.add_to(m)
m.add_child(folium.PolyLine(np.stack(poly.exterior.xy).T, color='yellow'))
mdf.apply(add_marker, axis=1)
m